# Section 25: Complete Data Science Project with Multiple Linear Regression

## Objectives
- Repeat the Mod 1 Project analysis in a streamlined way.

- Learn about `pandas_profiling` for quick and easy EDA (don't use on Mod Projects please).

- Intro to the idea of pipelines / programmatic construction of models

- Learn about [Variance Inflation Factor](https://etav.github.io/python/vif_factor_python.html) and how to use it to address multicollinearity. ( [Wikipedia: VIF](https://en.wikipedia.org/wiki/Variance_inflation_factor))




### Questions?
- Checking for normality and what to do about it.
- Checking how much scaling affects the output

### Follow Ups
- Question regarding removal high VIF variables and if the others that were identified as high VIF would still be high after removing an initial VIF>5.

# OBTAIN

In [12]:
import scipy.stats as stats
import statsmodels.api as sms
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
#
# import pandas_profiling


import warnings
warnings.filterwarnings('ignore')

In [13]:
# !pip install -U fsds
from fsds.imports import *
# Set pandas options
pd.set_option('display.precision',3)
pd.set_option('display.max_columns',0)

In [40]:
df = fs.datasets.load_ames_train(subset=True,read_csv_kwds={'index_col':0}).reset_index()#fs.datasets.load_mod1_proj()

df2 = fs.datasets.load_ames_train(subset=False,read_csv_kwds=dict(index_col=0))#.reset_index()#fs.datasets.load_mod1_proj()
bldg_house_style = df2.select_dtypes('object')[['BldgType','HouseStyle']].reset_index()

## Merge 
df = pd.merge(df,bldg_house_style,on='Id')
df.set_index('Id',inplace=True)
df.head()

,YrSold,MoSold,Fireplaces,TotRmsAbvGrd,GrLivArea,FullBath,YearRemodAdd,YearBuilt,OverallCond,OverallQual,LotArea,SalePrice,BldgType,HouseStyle
Id,,,,,,,,,,,,,,
1,2008,2,0,8,1710,2,2003,2003,5,7,8450,208500,1Fam,2Story
2,2007,5,1,6,1262,2,1976,1976,8,6,9600,181500,1Fam,1Story
3,2008,9,1,6,1786,2,2002,2001,5,7,11250,223500,1Fam,2Story
4,2006,2,1,7,1717,1,1970,1915,5,7,9550,140000,1Fam,2Story
5,2008,12,1,9,2198,2,2000,2000,5,8,14260,250000,1Fam,2Story


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   YrSold        1460 non-null   int64 
 1   MoSold        1460 non-null   int64 
 2   Fireplaces    1460 non-null   int64 
 3   TotRmsAbvGrd  1460 non-null   int64 
 4   GrLivArea     1460 non-null   int64 
 5   FullBath      1460 non-null   int64 
 6   YearRemodAdd  1460 non-null   int64 
 7   YearBuilt     1460 non-null   int64 
 8   OverallCond   1460 non-null   int64 
 9   OverallQual   1460 non-null   int64 
 10  LotArea       1460 non-null   int64 
 11  SalePrice     1460 non-null   int64 
 12  BldgType      1460 non-null   object
 13  HouseStyle    1460 non-null   object
dtypes: int64(12), object(2)
memory usage: 171.1+ KB


In [53]:
df['HouseStyle'].value_counts().index

Index(['1Story', '2Story', '1.5Fin', 'SLvl', 'SFoyer', '1.5Unf', '2.5Unf',
       '2.5Fin'],
      dtype='object')

In [51]:
df.select_dtypes('O')

,BldgType,HouseStyle
Id,,
1,OneFam,2Story
2,OneFam,1Story
3,OneFam,2Story
4,OneFam,2Story
5,OneFam,2Story
...,...,...
1456,OneFam,2Story
1457,OneFam,1Story
1458,OneFam,2Story


In [46]:
## Fix Col Names with Numbers
repl_dict = {'1Fam':'OneFam','2fmCon':'TwoFmCon'}

df['BldgType'] = df['BldgType'].replace(repl_dict)

In [47]:
df['BldgType'].value_counts()


OneFam      1220
TwnhsE       114
Duplex        52
Twnhs         43
TwoFmCon      31
Name: BldgType, dtype: int64

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   YrSold        1460 non-null   int64 
 1   MoSold        1460 non-null   int64 
 2   Fireplaces    1460 non-null   int64 
 3   TotRmsAbvGrd  1460 non-null   int64 
 4   GrLivArea     1460 non-null   int64 
 5   FullBath      1460 non-null   int64 
 6   YearRemodAdd  1460 non-null   int64 
 7   YearBuilt     1460 non-null   int64 
 8   OverallCond   1460 non-null   int64 
 9   OverallQual   1460 non-null   int64 
 10  LotArea       1460 non-null   int64 
 11  SalePrice     1460 non-null   int64 
 12  BldgType      1460 non-null   object
 13  HouseStyle    1460 non-null   object
dtypes: int64(12), object(2)
memory usage: 171.1+ KB


# SCRUB / EXPLORE

In [ ]:
drop_cols =['id','date','view']

df.drop(drop_cols, axis=1,inplace=True)
df.info()

In [ ]:
## Replacing Values
repl_dict = {'sqft_basement':('?','0.0')}

for col,replace in repl_dict.items():
    df[col] = df[col].replace(replace[0], replace[1])
    display(df[col].value_counts())

In [ ]:
## Recasting datatypes
recast_dict = {'sqft_basement':'float'}
for col,dtype in recast_dict.items():
    df[col] = df[col].astype(dtype)
df.dtypes

In [ ]:
## Fill Null values / zeros
fillna_dict = {'waterfront':0,
              'yr_renovated':0}

for col,val in fillna_dict.items():
    df[col].fillna(val,inplace=True)
    

In [ ]:
pandas_profiling.ProfileReport(df)

# Model 

In [ ]:
def make_ols_f(df,target='price',cat_cols = ['zipcode','grade'],
               col_list=None, show_summary=True,exclude_cols=[]):
    
    if col_list is None:
        col_list = list(df.drop(target,axis=1).columns)
        
    ## remove exclude cols
    [col_list.remove(ecol) for ecol in exclude_cols if ecol in col_list]

    features = '+'.join(col_list)


    for col in cat_cols:
        features = features.replace(col,f"C({col})")



    formula = target+'~'+features #target~predictors

    model = smf.ols(formula=formula, data=df).fit()
    
    if show_summary:
        display(model.summary())

    return model

## diagnostic function

def diagnose_model(model):
    resids = model.resid
    
    fig,ax = plt.subplots(ncols=2,figsize=(10,5))
    sms.qqplot(resids, stats.distributions.norm,
              fit=True, line='45',ax=ax[0])
    xs = np.linspace(0,1,len(resids))
    ax[1].scatter(x=xs,y=resids)
    
    return fig,ax 

model = make_ols_f(df)
diagnose_model(model)

# Interpret

- How does our model look? 
    - Clearly have not addressed normality 
        - First apply outlier removal  
        - Then try logging as a follow up
        
- What haven't we addressed yet?
    - Multicollinearity

# Revised Model 

In [ ]:
def find_outliers_Z(df,col):
    """Use scipy to calcualte absoliute Z-scores 
    and return boolean series where True indicates it is an outlier
    Args:
        col (Series): a series/column from your DataFrame
    Returns:
        idx_outliers (Series): series of  True/False for each row in col
        
    Ex:
    >> idx_outs = find_outliers(df['bedrooms'])
    >> df_clean = df.loc[idx_outs==False]"""
    from scipy import stats

    col = df[col]
    z = np.abs(stats.zscore(col))
    idx_outliers = np.where(z>3,True,False)
    return idx_outliers

def find_outliers_IQR(df,col):
    res = df[col].describe()
    IQR = res['75%'] -  res['25%']
    lower_limit = res['25%'] - 1.5*IQR
    upper_limit = res['75%'] + 1.5*IQR
    
    idx_goodvals = (df[col]<upper_limit) & (df[col]>lower_limit) 
    
    return ~idx_goodvals

In [ ]:
idx_outliers_z = find_outliers_Z(df,'bedrooms')
# idx_outliers.sum()
idx_outliers_z.sum()

In [ ]:
idx_outliers_IQR = find_outliers_IQR(df,'bedrooms')
# idx_outliers.sum()
idx_outliers_IQR.sum()

In [ ]:
df_outliers_Z = pd.DataFrame()
df_outliers_IQR = pd.DataFrame()

for col in df.columns:
    df_outliers_Z[col] = find_outliers_Z(df,col)
    df_outliers_IQR[col] = find_outliers_IQR(df,col)

idx_outs_IQR = df_outliers_IQR.any(axis=1)
idx_outs_z= df_outliers_Z.any(axis=1)

In [ ]:
print(idx_outs_z.sum()/len(idx_outs_z)*100)
print(idx_outs_IQR.sum()/len(idx_outs_IQR)*100)


In [ ]:
#drop zscore outleirs
df.loc[idx_outs_z==True].describe()

In [ ]:
df =df.loc[idx_outs_z ==False]
df.describe()

# Revised Model 2

**Work Tracker (How long did the project take me?):**
- Start: 12/10/19 ~8:45pm 
- End: 12/10/19 11:00 pm

In [ ]:
model2 = make_ols_f(df)
diagnose_model(model2)

# Explore 2: Multicollinearity with VIF

Definition: when 2 features are more related to each than the target.

> $\large V.I.F. = \frac{1}{(1 - R^2)} $<br>
- [VIF (Variance Inflation Factor)](https://etav.github.io/python/vif_factor_python.html) 
- [Wikipedia: VIF](https://en.wikipedia.org/wiki/Variance_inflation_factor)

In [ ]:
fs.ihelp(fs.ds.list2df)

In [ ]:

def vif_ols(df,exclude_col = None, cat_cols = ['grade','zipcode']):
    # let's check each column, build a model and get the r2
    vif_scores = [['Column','VIF','R2']]

    if exclude_col is not None:
        df = df.drop(exclude_col,axis=1)
        
    for column in df.columns:
        columns_to_use = df.drop(columns=[column]).columns
        target = column
        linreg = make_ols_f(df, target=target, cat_cols=cat_cols,
                            col_list=columns_to_use,show_summary=False)
        R2 = linreg.rsquared
        VIF = 1 / (1 - R2)
    #     print(f"VIF for {column} = {VIF}")
        vif_scores.append([column, VIF, R2])

    res = fs.ds.list2df(vif_scores,index_col='Column')
    res.sort_values('VIF',ascending=False,inplace=True)
    res['use']=res['VIF'] <5
    return res

res = vif_ols(df,exclude_col='price',)
res

In [ ]:
res2 = vif_ols(df,exclude_col=['price','sqft_above'])

In [ ]:
vif_scores = [['Column','VIF','R2']]
model_target= 'price'

for col in df.drop(model_target,axis=1).columns:
    
    columns_to_use = df.drop(columns=[col]).columns
    
    target=col
#     cat_cols = 
    linreg = make_ols_f(df,target,show_summary=False)
    R2 = linreg.rsquared
    
    VIF = 1/ (1-R2)
    
    vif_scores.append([col,VIF,R2])
    
res = fs.ds.list2df(vif_scores,index_col='Column')
res.sort_values('VIF',ascending=False,inplace=True)
res['use']=res['VIF'] <5
res

In [ ]:
# sns.heatmap(df.corr())
fs.jmi.multiplot(df)

In [ ]:
res[res['use']]

In [ ]:
df_use = df[res[res['use']].index]
df_use['price'] = df['price'].copy()

# Final Interpretations/Conclusions

In [ ]:
model3 = make_ols_f(df_use)
diagnose_model(model3)

In [ ]:
res2 = vif_ols(df,exclude_col=['price','sqft_above'])
df_use2 = df[res2[res2['use']].index]
df_use2['price'] = df['price'].copy()

In [ ]:
df_use2

In [ ]:
model4 = make_ols_f(df_use2)#,exclude_cols=['floors','lat','waterfront'])
diagnose_model(model4)

# Logging Vars 

In [ ]:
def check_normality(df,col):
    
    sns.distplot(df[col])
    plt.show()
    
    stat,p = stats.normaltest(df[col])
    
    return p

p_list = [['Column','Pval']]
for col in df.columns:
    p_list.append([col,check_normality(df,col)])
    
p_list = fs.ds.list2df(p_list)
p_list['reject'] = p_list['Pval']<0.05


In [ ]:
log_list=['price','sqft_living15','sqft_lot','sqft_lot15']
for col in log_list:
    df[col+'_log'] = np.log(df[col])
df.head()

In [ ]:
df_logged = df.copy()
df_logged.drop(log_list,axis=1,inplace=True)

In [ ]:
df_logged.describe()

In [ ]:
model5 = make_ols_f(df_logged,target='price_log')#,exclude_cols=['sqft_basement',
#                                                        'yr_built'])
diagnose_model(model5)

# Scaling

In [ ]:
df.describe()

In [ ]:
# cat_cols = ['zipcode','grade']
# cat_cols.extend([''])
df = df_logged.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# df_minmax = 
cat_cols = ['zipcode','grade']
cols_to_drop =cat_cols

cols_to_drop.append('price_log')
cols_to_scale = df.drop(cols_to_drop,axis=1).columns
# cols_to


scaler = MinMaxScaler()
minmax_data = scaler.fit_transform(df[cols_to_scale])

df_minmax = pd.DataFrame(minmax_data,columns=cols_to_scale)
df_minmax['price_log'] = df['price_log'].copy()
df_minmax['zipcode']= df['zipcode']
df_minmax['grade']= df['grade']

df_minmax.describe()

In [ ]:
model_default = make_ols_f(df_logged,target='price_log',cat_cols=cat_cols)
diagnose_model(model_default)

In [ ]:
model_minmax = make_ols_f(df_minmax,target='price_log',cat_cols=cat_cols)
diagnose_model(model_minmax)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# df_minmax = 
cat_cols = ['zipcode','grade']
cols_to_drop =cat_cols

cols_to_drop.append('price_log')
cols_to_scale = df.drop(cols_to_drop,axis=1).columns
# cols_to


scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols_to_scale])

df_standard = pd.DataFrame(scaled_data,columns=cols_to_scale)
df_standard['price_log'] = df['price_log'].copy()
df_standard['zipcode']= df['zipcode']
df_standard['grade']= df['grade']

df_standard.describe().round(3)

In [ ]:
model_standard = make_ols_f(df_standard,target='price_log',cat_cols=cat_cols)
diagnose_model(model_standard)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# df_minmax = 
cat_cols = ['zipcode','grade']
cols_to_drop =cat_cols

# cols_to_drop.append('price_log')
cols_to_scale = df.drop(cols_to_drop,axis=1).columns
# cols_to


scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols_to_scale])

df_standard = pd.DataFrame(scaled_data,columns=cols_to_scale)
# df_standard['price_log'] = df['price_log'].copy()
df_standard['zipcode']= df['zipcode']
df_standard['grade']= df['grade']

df_standard.describe().round(3)

In [ ]:
model_standard = make_ols_f(df_standard,target='price_log',
                            exclude_cols=['zipcode','grade','sqft_lot_log'])#cat_cols=cat_cols)
diagnose_model(model_standard)

# Conclusion
- Scaling is not necessarily as helpful as dogma would indicate (at least with statsmodels).
- VIF identified new columns as multicoll that heatmap correlation did not indicate (according to pairwise comparisons). 
- log transformation was indeed helpful (though slightly complicates interpretation)

- IQR outier removal is more potent/liberal than using Z-scores
    - Did not check if IQR outlier removal would have provided better results.
    - We could not apply IQR across the board without removing all of data